In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

## 1. 데이터 불러오기 (2021 기준)
    - 1-1. 휴장일
    - 1-2. Newsfile 
    - 1-3. 반도체(005290) : 상승률 1.5 이상--1, 상승률 1.5 이하--0 라벨링 데이터

### 1-1. 휴장일 날짜

In [2]:
closedate = pd.read_csv('../data/closedate_all.csv')
closedate.head(3)

,Unnamed: 0,일자 및 요일,요일구분,비고
0,0,2015-01-01,목요일,신정
1,1,2015-02-18,수요일,설날
2,2,2015-02-19,목요일,설날


In [3]:
closedate = closedate[['일자 및 요일', '요일구분']]
closedate.head(3)

,일자 및 요일,요일구분
0,2015-01-01,목요일
1,2015-02-18,수요일
2,2015-02-19,목요일


In [4]:
closedate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   일자 및 요일  104 non-null    object
 1   요일구분     104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [5]:
closedate = closedate.astype({'일자 및 요일':'string'})
closedate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   일자 및 요일  104 non-null    string
 1   요일구분     104 non-null    object
dtypes: object(1), string(1)
memory usage: 1.8+ KB


- 2021 date

In [6]:
closedate['일자 및 요일'] = pd.to_datetime(closedate['일자 및 요일'])
closedate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   일자 및 요일  104 non-null    datetime64[ns]
 1   요일구분     104 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1.8+ KB


In [7]:
closedate_2021 = closedate[closedate['일자 및 요일'].between('2021-01-01','2021-12-31')]
closedate_2021

,일자 및 요일,요일구분
91,2021-01-01,금요일
92,2021-02-11,목요일
93,2021-02-12,금요일
94,2021-03-01,월요일
95,2021-05-05,수요일
96,2021-05-19,수요일
97,2021-08-16,월요일
98,2021-09-20,월요일
99,2021-09-21,화요일
100,2021-09-22,수요일


- 컬럼명 변경

In [8]:
closedate_2021.rename(columns={'일자 및 요일':'휴장일'}, inplace=True)
closedate_2021

,휴장일,요일구분
91,2021-01-01,금요일
92,2021-02-11,목요일
93,2021-02-12,금요일
94,2021-03-01,월요일
95,2021-05-05,수요일
96,2021-05-19,수요일
97,2021-08-16,월요일
98,2021-09-20,월요일
99,2021-09-21,화요일
100,2021-09-22,수요일


### 1-2. newsfile

- train_data

In [9]:
news_train = pd.read_csv('../data/train_data_hanja_translate.csv')
news_train.head(3)

,Unnamed: 0,date,title,nouns
0,39493,20181030,삼성 반도체 통합 출범 년매출 배 영업익 배,삼성 반도체 통합 출범 년 매출 배 영업 익 배
1,25343,20200204,현대차 수소차 판매 위 질주SK 반도체 수직계열화 완성,현대차 수소차 판매 위 질주 반도체 수직 계열 완성
2,53672,20161225,내년 메모리반도체 시장 삼성전자SK하이닉스 탄탄대로,내년 메모리 반도체 시장 삼성 전자 하이닉스 탄탄


In [10]:
news_train = news_train[['date', 'title', 'nouns']]
news_train.head(3)

,date,title,nouns
0,20181030,삼성 반도체 통합 출범 년매출 배 영업익 배,삼성 반도체 통합 출범 년 매출 배 영업 익 배
1,20200204,현대차 수소차 판매 위 질주SK 반도체 수직계열화 완성,현대차 수소차 판매 위 질주 반도체 수직 계열 완성
2,20161225,내년 메모리반도체 시장 삼성전자SK하이닉스 탄탄대로,내년 메모리 반도체 시장 삼성 전자 하이닉스 탄탄


- test_data

In [11]:
news_test = pd.read_csv('../data/test_data_hanja_translate.csv')
news_test.head(3)

,Unnamed: 0,date,title,nouns
0,24519,20200323,일 수출 증가반도체승용차 등 늘어,일 수출 증가 반도체 승용차 등
1,25070,20200217,신종 코로나에도 끄떡없는 한반도체,신종 코로나 떡 반도체
2,29830,20190808,일경산성 반도체 소재 건 수출허가 배경은,산성 반도체 소재 건 수출 허가 배경


In [12]:
news_test = news_test[['date', 'title', 'nouns']]
news_test.head(3)

,date,title,nouns
0,20200323,일 수출 증가반도체승용차 등 늘어,일 수출 증가 반도체 승용차 등
1,20200217,신종 코로나에도 끄떡없는 한반도체,신종 코로나 떡 반도체
2,20190808,일경산성 반도체 소재 건 수출허가 배경은,산성 반도체 소재 건 수출 허가 배경


- concat

In [13]:
news_data = pd.concat([news_train, news_test])
news_data.head(10)

,date,title,nouns
0,20181030,삼성 반도체 통합 출범 년매출 배 영업익 배,삼성 반도체 통합 출범 년 매출 배 영업 익 배
1,20200204,현대차 수소차 판매 위 질주SK 반도체 수직계열화 완성,현대차 수소차 판매 위 질주 반도체 수직 계열 완성
2,20161225,내년 메모리반도체 시장 삼성전자SK하이닉스 탄탄대로,내년 메모리 반도체 시장 삼성 전자 하이닉스 탄탄
3,20200702,루비오 등 미상원의원 명 첨단 반도체 제조업 미로 가져오라,루비 오 등 상원의원 명 첨단 반도체 제조업 미로
4,20190405,삼성전자 영업익 추락 분기에도 반도체 비관론 삼성은 초격차로 위기,삼성 전자 영업 익 추락 분기 반도체 비관 론 삼성 초 격차 위기
5,20180622,베스트리포트아이원스 글로벌 반도체 부품기업 도약,베스트 리포트 아이 원스 글로벌 반도체 부품 기업 도약
6,20210614,현대차 앨라배마 공장 결국 스톱반도체 부족에 주일 중단,현대차 앨라배마 공장 스톱 반도체 부족 주일 중단
7,20210521,메모리 반도체 호황기 진입한다내년 사상 최대 매출 예상,메모리 반도체 호황 기 진입 내년 사상 최대 매출 예상
8,20190311,한미반도체 SK하이닉스와 억원 규모 공급계약 체결,한미 반도체 하이닉스 억원 규모 공급 계약 체결
9,20170515,KCC 반도체 소재 공략 한창,반도체 소재 공략 한창


- 날짜순 정렬

In [14]:
news_data = news_data.sort_values(by="date")
news_data.head(10)

,date,title,nouns
5453,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
19712,20151019,세계 반도체 MA 억달러,세계 반도체 달러
14154,20151019,제주반도체 해외투자유치설 조회공시 요구,제주 반도체 해외 투자 유치 설 조회 공시 요구
6781,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
695,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상
12612,20151019,반도체사 인수합병 봇물올 들어 억달러 돌파,반도 체사 인수합병 봇 물올 달러 돌파
22237,20151020,STS반도체 김영민 씨로 대표이사 변경,반도체 김영민 씨 대표이사 변경
29251,20151020,공시 제주반도체 종속회사 램스웨이 만 달러 투자 유치,공시 제주 반도체 종 회사 램 스웨 만 달러 투자 유치
1378,20151020,전자부품업계 분기 실적 전망은 반도체디스플레이 패널가 급락 우울,전자부품 업계 분기 실적 전망 반도체 디스플레이 패널 급락 우울
407,20151020,투자유치 제주반도체월 유증 데쟈뷰,투자 유치 제주 반도체 월 유증 데쟈뷰


In [15]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 5453 to 1719
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [16]:
news_data = news_data.astype({'date':'string'})
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 5453 to 1719
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: object(2), string(1)
memory usage: 1.8+ MB


In [17]:
news_data.to_csv('Newsfile_nouns.csv')

- 2021년도 date

In [18]:
# Datetime 형식으로 변환
news_data['date'] = pd.to_datetime(news_data['date'])
news_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 5453 to 1719
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58442 non-null  datetime64[ns]
 1   title   58442 non-null  object        
 2   nouns   58442 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1.8+ MB


In [19]:
news_data_2021 = news_data[news_data['date'].between('2021-01-01','2021-12-31')]
news_data_2021

,date,title,nouns
27927,2021-01-01,증시 진단 전문가 원픽은 삼전 배터리반도체친환경 뜬다,증시 진단 전문가 원픽 삼전 배터리 반도체 친환경
29049,2021-01-01,새해 첫 수출 화물은 반도체 목적지는 중국,새해 첫 수출 화물 반도체 목적 중국
11915,2021-01-01,역대급 실적 월 수출 억 달러반도체 상보,역대 실적 월 수출 억 달러 반도체 상보
14637,2021-01-01,반도체디스플레이 활약에 두자릿수 수출펑크 겨우막아,반도체 디스플레이 활약 자릿수 수출 펑크 겨우
28898,2021-01-01,아시아나항공 새해 첫 수출 화물은 반도체중국 운송,아시아나 항공 새해 첫 수출 화물 반도체 중국 운송
...,...,...,...
15527,2021-12-31,삼성 이어 미 마이크론도시안 봉쇄 반도체 공정에 영향,삼성 마이크론 시안 봉쇄 반도체 공정 영향
30025,2021-12-31,월 D램낸드 등 메모리반도체 가격 제자리,월 램 낸드 등 메모리 반도체 가격 제자리
5940,2021-12-31,테슬라 반도체 공급부족에도 역대급 생산이유는,테슬라 반도체 공급 부족 역대 생산 이유
23468,2021-12-31,삼성 중 시안 반도체 생산 차질 대규모 감산 없을 것,삼성 중 시안 반도체 생산 차질 대규모 감산 것


## 1-3. 반도체(005290) 상승률 기준 라벨링 데이터

In [25]:
updown = pd.read_excel('../data/반도체주가데이터/반도체주가데이터/동진쎄미캠.xlsx')
updown.head(3)

,Unnamed: 0,Unnamed: 0.1,일자,현재가,당일종가-어제종가,상승률,label
0,0,0,20220314,31900,-1500,-0.044910,0
1,1,1,20220311,33400,-300,-0.008902,0
2,2,2,20220310,33700,550,0.016591,1


In [26]:
updown.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
updown.head(3)

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,31900,-1500,-0.044910,0
1,20220311,33400,-300,-0.008902,0
2,20220310,33700,550,0.016591,1


In [27]:
updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   일자         5484 non-null   int64  
 1   현재가        5484 non-null   int64  
 2   당일종가-어제종가  5484 non-null   int64  
 3   상승률        5484 non-null   float64
 4   label      5484 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 214.3 KB


In [28]:
updown = updown.astype({'일자':'string'})
updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   일자         5484 non-null   string 
 1   현재가        5484 non-null   int64  
 2   당일종가-어제종가  5484 non-null   int64  
 3   상승률        5484 non-null   float64
 4   label      5484 non-null   int64  
dtypes: float64(1), int64(3), string(1)
memory usage: 214.3 KB


- 2021년도 date

In [29]:
updown['일자'] = pd.to_datetime(updown['일자'])
updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   일자         5484 non-null   datetime64[ns]
 1   현재가        5484 non-null   int64         
 2   당일종가-어제종가  5484 non-null   int64         
 3   상승률        5484 non-null   float64       
 4   label      5484 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 214.3 KB


In [30]:
updown_2021 = updown[updown['일자'].between('2021-01-01','2021-12-31')]
updown_2021

,일자,현재가,당일종가-어제종가,상승률,label
46,2021-12-30,51000,6450,0.144781,1
47,2021-12-29,44550,-950,-0.020879,0
48,2021-12-28,45500,950,0.021324,1
49,2021-12-27,44550,-800,-0.017641,0
50,2021-12-24,45350,2500,0.058343,1
...,...,...,...,...,...
289,2021-01-08,35900,250,0.007013,0
290,2021-01-07,35650,150,0.004225,0
291,2021-01-06,35500,-1100,-0.030055,0
292,2021-01-05,36600,-400,-0.010811,0


In [31]:
updown_2021.sort_values('일자')

,일자,현재가,당일종가-어제종가,상승률,label
293,2021-01-04,37000,500,0.013699,0
292,2021-01-05,36600,-400,-0.010811,0
291,2021-01-06,35500,-1100,-0.030055,0
290,2021-01-07,35650,150,0.004225,0
289,2021-01-08,35900,250,0.007013,0
...,...,...,...,...,...
50,2021-12-24,45350,2500,0.058343,1
49,2021-12-27,44550,-800,-0.017641,0
48,2021-12-28,45500,950,0.021324,1
47,2021-12-29,44550,-950,-0.020879,0
